In [6]:
import os
import sys
import argparse
import csv
import pandas as pd
import datetime as dt
import numpy as np
from pprint import pprint
from tqdm import tqdm
from pytictoc import TicToc
import traceback
from statistics import median
from statistics import mean
from statistics import mode
from statistics import stdev
from scipy import stats
from scipy import signal
import portion as P
import math
import random
import itertools as it

pd.set_option('display.max_columns', 200)
# pd.set_option('display.max_rows', 200)

def makedir(dirpath, mode=0):  # mode=1: show message, mode=0: hide message
    if os.path.isdir(dirpath):
        if mode:
            print("mkdir: cannot create directory '{}': directory has already existed.".format(dirpath))
        return
    ### recursively make directory
    _temp = []
    while not os.path.isdir(dirpath):
        _temp.append(dirpath)
        dirpath = os.path.dirname(dirpath)
    while _temp:
        dirpath = _temp.pop()
        print("mkdir", dirpath)
        os.mkdir(dirpath)

sorter = ['stable',
          'LTE_HO','MN_HO','SN_HO','MNSN_HO','SN_Rel','SN_Setup','SN_Rel_MN_HO','SN_Setup_MN_HO',
          'SCG_Failure_t310-Expiry (0)','SCG_Failure_randomAccessProblem (1)','SCG_Failure_rlc-MaxNumRetx (2)','SCG_Failure_synchReconfigFailureSCG (3)',
          'SCG_Failure_scg-ReconfigFailure (4)','SCG_Failure_srb3-IntegrityFailure (5)','SCG_Failure_other-r16 (6)',
          'MCG_Failure_reconfigurationFailure (0)','MCG_Failure_handoverFailure (1)','MCG_Failure_otherFailure (2)'
          'NAS_Recovery_reconfigurationFailure (0)','NAS_Recovery_handoverFailure (1)','NAS_Recovery_otherFailure (2)']
# mcg failure types: https://www.sharetechnote.com/html/Handbook_LTE_RRC_ConnectionReestablishment.html (0)(1)(2)
# scg failure types: https://www.sharetechnote.com/html/5G/5G_Release17.html (1)(3)

# Define Functions

In [7]:
def align_data(dfs):
    st, et = [], []
    for schm in dfs.keys():
        st.append(dfs[schm]['seq'].array[0])
        et.append(dfs[schm]['seq'].array[-1])
    st, et = max(st), min(et)
    for schm in dfs.keys():
        dfs[schm] = dfs[schm][(dfs[schm]['seq'] >= st) & (dfs[schm]['seq'] <= et)].reset_index(drop=True)
        dfs[schm]['ho_type0'] = dfs[schm]['ho_type0'].cat.set_categories(sorter)
        # for col in dfs[schm].columns[2:]:
        #     dfs[schm] = dfs[schm].rename(columns={col:f'{col}_{schm}'})
    return dfs

def consolidate_data(dfs, schemes):
    xs = list(it.combinations(range(len(schemes)), 2))
    xs = [(schemes[s1], schemes[s2]) for (s1, s2) in xs]
    for schm1, schm2 in xs:
        df1, df2 = dfs[schm1].copy(), dfs[schm2].copy()
        key = f'{schm1}+{schm2}'
        # dfs[key] = pd.merge(df1, df2, on=['seq','Timestamp'])
        dfs[key] = pd.merge(df1, df2, on=['seq','Timestamp'], suffixes=(f'_{schm1}', f'_{schm2}'))
        dfs[key][f'lost_{key}'] = dfs[key][f'lost_{schm1}'] & dfs[key][f'lost_{schm2}']
        dfs[key][f'lost_{schm1}+^{schm2}'] = dfs[key][f'lost_{schm1}'] & ~dfs[key][f'lost_{schm2}']
        dfs[key][f'lost_^{schm1}+{schm2}'] = ~dfs[key][f'lost_{schm1}'] & dfs[key][f'lost_{schm2}']
        dfs[key][f'lost_^{schm1}+^{schm2}'] = ~(dfs[key][f'lost_{schm1}'] | dfs[key][f'lost_{schm2}'])
        dfs[key][f'excl_{key}'] = dfs[key][f'excl_{schm1}'] & dfs[key][f'excl_{schm2}']
        dfs[key][f'excl_{schm1}+^{schm2}'] = dfs[key][f'excl_{schm1}'] & ~dfs[key][f'excl_{schm2}']
        dfs[key][f'excl_^{schm1}+{schm2}'] = ~dfs[key][f'excl_{schm1}'] & dfs[key][f'excl_{schm2}']
        dfs[key][f'excl_^{schm1}+^{schm2}'] = ~(dfs[key][f'excl_{schm1}'] | dfs[key][f'excl_{schm2}'])
        dfs[key][f'latency_{key}'] = dfs[key][[f'latency_{schm1}', f'latency_{schm2}']].min(axis=1)
    return dfs

# Testing

In [8]:
# define settings
devices = ['qc01','qc02','qc03']
schemes = ['B3','B7','B8']

# read data
dfs = {}
testpath = [
    '/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04 #1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc01/#01/data/udp_dnlk_loss_latency_ho.pkl',
    '/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04 #1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc02/#01/data/udp_dnlk_loss_latency_ho.pkl',
    '/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04 #1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc03/#01/data/udp_dnlk_loss_latency_ho.pkl',
]
for i, (dev, schm) in enumerate(zip(devices, schemes)):
    dfs[schm] = pd.read_pickle(testpath[i])

# consolidate data & output
dfs = align_data(dfs)
dfs = consolidate_data(dfs, schemes)
for schm, df in dfs.items():
    df.to_pickle(f'./example/udp_dnlk_loss_latency_ho_{schm}.pkl')
    # display(df)

# testing
xs = list(it.combinations(range(len(schemes)), 2))
xs = [(schemes[s1], schemes[s2]) for (s1, s2) in xs]
xsk = [f'{s1}+{s2}' for (s1, s2) in xs]
print([*schemes, *xsk])

['B3', 'B7', 'B8', 'B3+B7', 'B3+B8', 'B7+B8']


# Inspect Data

In [9]:
# df = pd.read_pickle("/Users/jackbedford/Desktop/MOXA/Code/wmnl-handoff-research/analysis-beta/example/udp_dnlk_loss_latency_ho_B7+B8.pkl")
df = dfs['B3+B8'].copy()

In [10]:
df['ho_type0_B3'].value_counts().index

CategoricalIndex(['stable', 'MN_HO', 'SN_HO', 'SN_Setup', 'SN_Rel', 'LTE_HO',
                  'SCG_Failure_scg-ReconfigFailure (4)',
                  'NAS_Recovery_handoverFailure (1)',
                  'MCG_Failure_otherFailure (2)NAS_Recovery_reconfigurationFailure (0)',
                  'MCG_Failure_handoverFailure (1)',
                  'MCG_Failure_reconfigurationFailure (0)',
                  'SCG_Failure_other-r16 (6)',
                  'SCG_Failure_srb3-IntegrityFailure (5)',
                  'SCG_Failure_randomAccessProblem (1)',
                  'SCG_Failure_synchReconfigFailureSCG (3)',
                  'SCG_Failure_rlc-MaxNumRetx (2)',
                  'SCG_Failure_t310-Expiry (0)', 'SN_Setup_MN_HO',
                  'SN_Rel_MN_HO', 'MNSN_HO', 'NAS_Recovery_otherFailure (2)'],
                 categories=['stable', 'LTE_HO', 'MN_HO', 'SN_HO', ..., 'MCG_Failure_handoverFailure (1)', 'MCG_Failure_otherFailure (2)NAS_Recovery_reco..., 'NAS_Recovery_handoverFailure

In [11]:
df['ho_type0_B3'].value_counts()

stable                                                                 230712
MN_HO                                                                   15295
SN_HO                                                                   12299
SN_Setup                                                                 3216
SN_Rel                                                                   1211
LTE_HO                                                                    706
SCG_Failure_scg-ReconfigFailure (4)                                         0
NAS_Recovery_handoverFailure (1)                                            0
MCG_Failure_otherFailure (2)NAS_Recovery_reconfigurationFailure (0)         0
MCG_Failure_handoverFailure (1)                                             0
MCG_Failure_reconfigurationFailure (0)                                      0
SCG_Failure_other-r16 (6)                                                   0
SCG_Failure_srb3-IntegrityFailure (5)                           

## Single Radio Loss

### classify by ho_type0

In [12]:
# specify a scheme
schemes = ['B3','B8']
schm = schemes[0]

# count loss
table = df.loc[df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{schm}'})

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'lost_{schm}'] = table[f'lost_{schm}'].astype('Int16')

# calculate loss rate
table[f'lorate(%)_{schm}'] = round(table[f'lost_{schm}'] / table[f'total_{schm}'] * 100, 3)

# show table
table
table[table[f'lost_{schm}'] != 0]

,ho_type0_B3,lost_B3,total_B3,lorate(%)_B3
0,stable,3,230712,0.001
2,MN_HO,333,15295,2.177
6,SN_Setup,117,3216,3.638


### classify by ho_type0 & ho_index

In [13]:
# specify a scheme
schemes = ['B3','B8']
schm = schemes[0]

# count loss
table = df.loc[df[f'lost_{schm}'], [f'ho_type0_{schm}', f'ho_index_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{schm}'})

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm}', f'ho_index_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}', f'ho_index_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}', f'ho_index_{schm}']).reset_index(drop=True).fillna(0)
table[f'lost_{schm}'] = table[f'lost_{schm}'].astype('Int16')

# calculate loss rate
table[f'lorate(%)_{schm}'] = round(table[f'lost_{schm}'] / table[f'total_{schm}'] * 100, 3)

# event count
table[f'evt_count_{schm}'] = pd.Series(dtype='Int16')
tb_dict = table[f'ho_type0_{schm}'].value_counts().to_dict()
for key, val in tb_dict.items():
    table.loc[table[f'ho_type0_{schm}'] == key, f'evt_count_{schm}'] = val

# show table
table
table[table[f'lost_{schm}'] != 0]

,ho_type0_B3,ho_index_B3,lost_B3,total_B3,lorate(%)_B3,evt_count_B3
0,stable,-1,3,230712,0.001,1
8,MN_HO,7,332,1011,32.839,18
18,MN_HO,39,1,948,0.105,18
43,SN_Setup,25,117,523,22.371,6


In [14]:
# groupby
tb_group = table.groupby([f'ho_type0_{schm}'])

# aggregate data statistics
tb_lost = tb_group[f'lost_{schm}'].agg(['sum','mean']).reset_index().rename(columns={'sum':f'lost_{schm}', 'mean':f'lost_per_evt_{schm}'})
tb_total = tb_group[f'total_{schm}'].agg(['sum']).reset_index().rename(columns={'sum':f'total_{schm}'})
tb_lorate = tb_group[f'lorate(%)_{schm}'].agg(['mean']).round(3).reset_index().rename(columns={'mean':f'lorate_per_evt(%)_{schm}'})
ev_count = tb_group[f'ho_index_{schm}'].agg(['count']).reset_index().rename(columns={'count':f'evt_count_{schm}'})

# merge table
table_ = tb_lost.merge(tb_total, on=[f'ho_type0_{schm}'], how='right')
table_ = table_.merge(tb_lorate, on=[f'ho_type0_{schm}'], how='left')
table_ = table_.merge(ev_count, on=[f'ho_type0_{schm}'], how='left')
table_[f'lorate(%)_{schm}'] = round(table_[f'lost_{schm}'] / table_[f'total_{schm}'] * 100, 3)
table_[f'lost_per_evt_{schm}'] = table_[f'lost_per_evt_{schm}'].round(2)

# show table
table_
table_[table_[f'evt_count_{schm}'] != 0]

,ho_type0_B3,lost_B3,lost_per_evt_B3,total_B3,lorate_per_evt(%)_B3,evt_count_B3,lorate(%)_B3
0,stable,3,3.0,230712,0.001,1,0.001
1,LTE_HO,0,0.0,706,0.0,2,0.0
2,MN_HO,333,18.5,15295,1.83,18,2.177
3,SN_HO,0,0.0,12299,0.0,15,0.0
5,SN_Rel,0,0.0,1211,0.0,3,0.0
6,SN_Setup,117,19.5,3216,3.728,6,3.638


## Single Radio Excessive Latency

### classify by ho_type0

In [15]:
# specify a scheme
schemes = ['B3','B8']
schm = schemes[0]

# count loss
table = df.loc[~df[f'lost_{schm}'] & df[f'excl_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'excl_{schm}'})

# count recv packet
table1 = df.loc[~df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'recv_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'excl_{schm}'] = table[f'excl_{schm}'].astype('Int16')

# calculate excessive latency rate
table[f'exrate(%)_{schm}'] = round(table[f'excl_{schm}'] / table[f'recv_{schm}'] * 100, 3)

# show table
table
table[table[f'excl_{schm}'] != 0]

,ho_type0_B3,excl_B3,recv_B3,exrate(%)_B3
0,stable,221,230709,0.096
2,MN_HO,261,14962,1.744
5,SN_Rel,3,1211,0.248
6,SN_Setup,338,3099,10.907


### classify by ho_type0 & ho_index

In [16]:
# specify a scheme
schemes = ['B3','B8']
schm = schemes[0]

# count loss
table = df.loc[~df[f'lost_{schm}'] & df[f'excl_{schm}'], [f'ho_type0_{schm}', f'ho_index_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'excl_{schm}'})

# count recv packet
table1 = df.loc[~df[f'lost_{schm}'], [f'ho_type0_{schm}', f'ho_index_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'recv_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}', f'ho_index_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}', f'ho_index_{schm}']).reset_index(drop=True).fillna(0)
table[f'excl_{schm}'] = table[f'excl_{schm}'].astype('Int16')

# calculate excessive latency rate
table[f'exrate(%)_{schm}'] = round(table[f'excl_{schm}'] / table[f'recv_{schm}'] * 100, 3)

# event count
table[f'evt_count_{schm}'] = pd.Series(dtype='Int16')
tb_dict = table[f'ho_type0_{schm}'].value_counts().to_dict()
for key, val in tb_dict.items():
    table.loc[table[f'ho_type0_{schm}'] == key, f'evt_count_{schm}'] = val

# show table
table
table[table[f'excl_{schm}'] != 0]

,ho_type0_B3,ho_index_B3,excl_B3,recv_B3,exrate(%)_B3,evt_count_B3
0,stable,-1,221,230709,0.096,1
4,MN_HO,1,5,1009,0.496,18
6,MN_HO,3,11,1009,1.09,18
12,MN_HO,29,11,549,2.004,18
15,MN_HO,34,24,1010,2.376,18
16,MN_HO,35,13,1008,1.29,18
17,MN_HO,37,11,1007,1.092,18
18,MN_HO,39,58,947,6.125,18
19,MN_HO,40,20,953,2.099,18
20,MN_HO,43,108,574,18.815,18


In [17]:
# groupby
tb_group = table.groupby([f'ho_type0_{schm}'])

# aggregate data statistics
tb_excl = tb_group[f'excl_{schm}'].agg(['sum','mean']).reset_index().rename(columns={'sum':f'excl_{schm}', 'mean':f'excl_per_evt_{schm}'})
tb_recv = tb_group[f'recv_{schm}'].agg(['sum']).reset_index().rename(columns={'sum':f'recv_{schm}'})
tb_exrate = tb_group[f'exrate(%)_{schm}'].agg(['mean']).round(3).reset_index().rename(columns={'mean':f'exrate_per_evt(%)_{schm}'})
ev_count = tb_group[f'ho_index_{schm}'].agg(['count']).reset_index().rename(columns={'count':f'evt_count_{schm}'})

# merge table
table_ = tb_excl.merge(tb_recv, on=[f'ho_type0_{schm}'], how='right')
table_ = table_.merge(tb_exrate, on=[f'ho_type0_{schm}'], how='left')
table_ = table_.merge(ev_count, on=[f'ho_type0_{schm}'], how='left')
table_[f'exrate(%)_{schm}'] = round(table_[f'excl_{schm}'] / table_[f'recv_{schm}'] * 100, 3)
table_[f'excl_per_evt_{schm}'] = table_[f'excl_per_evt_{schm}'].round(2)

# show table
table_
table_[table_[f'evt_count_{schm}'] != 0]

,ho_type0_B3,excl_B3,excl_per_evt_B3,recv_B3,exrate_per_evt(%)_B3,evt_count_B3,exrate(%)_B3
0,stable,221,221.0,230709,0.096,1,0.096
1,LTE_HO,0,0.0,706,0.0,2,0.0
2,MN_HO,261,14.5,14962,1.966,18,1.744
3,SN_HO,0,0.0,12299,0.0,15,0.0
5,SN_Rel,3,1.0,1211,0.326,3,0.248
6,SN_Setup,338,56.33,3099,10.244,6,10.907


## Single Radio Latency

### classify by ho_type0

In [18]:
# specify a scheme
schemes = ['B3','B8']
schm = schemes[0]

# groupby & aggregate latency statistics
tb_group = df[~df[f'lost_{schm}']].groupby([f'ho_type0_{schm}'])
table = tb_group[f'latency_{schm}'].agg(['mean','median','min','max','count']).reset_index() \
    .rename(columns={'mean':f'mean_lat_{schm}','median':f'median_lat_{schm}','min':f'min_lat_{schm}','max':f'max_lat_{schm}','count':f'recv_{schm}'})

# show table
table
table[table[f'recv_{schm}'] != 0]

,ho_type0_B3,mean_lat_B3,median_lat_B3,min_lat_B3,max_lat_B3,recv_B3
0,stable,0.029646,0.029072,0.005484,0.359905,230709
1,LTE_HO,0.033653,0.034199,0.026240,0.042231,706
2,MN_HO,0.036325,0.035342,0.013727,0.246799,14962
3,SN_HO,0.038483,0.036927,0.022069,0.097155,12299
5,SN_Rel,0.033235,0.031105,0.018325,0.100544,1211
6,SN_Setup,0.052728,0.033542,0.018366,0.368318,3099


### classify by ho_type0 & ho_index

In [19]:
# specify a scheme
schemes = ['B3','B8']
schm = schemes[0]

# calculate first-order difference of latency
df[f'diff_{schm}'] = df[f'latency_{schm}'].diff().abs()

# groupby & aggregate latency statistics
tb_group = df[~df[f'lost_{schm}']].groupby([f'ho_type0_{schm}', f'ho_index_{schm}'])
table = tb_group[f'latency_{schm}'].agg(['mean','median','min','max','std','var','count']).dropna().reset_index() \
    .rename(columns={'mean':f'mean_lat_{schm}','median':f'median_lat_{schm}','min':f'min_lat_{schm}','max':f'max_lat_{schm}','std':f'std_lat_{schm}','var':f'var_lat_{schm}','count':f'recv_{schm}'})

# calculate jitter during each event
tb_group = df[~df[f'lost_{schm}'] & df[f'diff_{schm}'].notna()].groupby([f'ho_type0_{schm}', f'ho_index_{schm}'])
table1 = tb_group[f'diff_{schm}'].agg(['mean']).dropna().reset_index() \
    .rename(columns={'mean':f'jitter_{schm}'})
table = table.merge(table1, on=[f'ho_type0_{schm}', f'ho_index_{schm}'], how='left')

# event count
table[f'evt_count_{schm}'] = pd.Series(dtype='Int16')
tb_dict = table[f'ho_type0_{schm}'].value_counts().to_dict()
for key, val in tb_dict.items():
    table.loc[table[f'ho_type0_{schm}'] == key, f'evt_count_{schm}'] = val

# show table
table
table[table[f'recv_{schm}'] != 0]

,ho_type0_B3,ho_index_B3,mean_lat_B3,median_lat_B3,min_lat_B3,max_lat_B3,std_lat_B3,var_lat_B3,recv_B3,jitter_B3,evt_count_B3
0,stable,-1,0.029646,0.029072,0.005484,0.359905,0.014327,0.000205,230709,NaN,1
1,LTE_HO,13,0.029177,0.028308,0.026240,0.032315,0.001917,0.000004,128,0.002641,2
2,LTE_HO,26,0.034644,0.034217,0.030182,0.042231,0.002484,0.000006,578,0.002658,2
3,MN_HO,0,0.018885,0.018011,0.013727,0.078085,0.007974,0.000064,1006,0.002694,18
4,MN_HO,1,0.021915,0.019028,0.014842,0.106865,0.011454,0.000131,1009,0.002707,18
5,MN_HO,2,0.023140,0.021401,0.017090,0.097469,0.011740,0.000138,1009,0.002703,18
6,MN_HO,3,0.026155,0.022285,0.018177,0.114251,0.013912,0.000194,1009,0.002720,18
7,MN_HO,4,0.022067,0.022269,0.017991,0.028433,0.002435,0.000006,684,0.002645,18
8,MN_HO,7,0.022067,0.022453,0.018422,0.028552,0.002215,0.000005,679,NaN,18
9,MN_HO,8,0.027643,0.025963,0.021908,0.085983,0.008494,0.000072,1006,0.002699,18


In [20]:
def weighted_average(df, values, weights):
    return sum(df[weights] * df[values]) / df[weights].sum()

# groupby
tb_group = table.groupby([f'ho_type0_{schm}'])
table_ = tb_group[f'ho_index_{schm}'].agg(['count']).reset_index().rename(columns={'count':f'evt_count_{schm}'})

# latency statistics
for key in [f'mean_lat_{schm}', f'median_lat_{schm}', f'min_lat_{schm}', f'max_lat_{schm}', f'std_lat_{schm}', f'var_lat_{schm}']:
    tb_ss = table.groupby(f'ho_type0_{schm}').apply(weighted_average, key, f'evt_count_{schm}').reset_index().rename(columns={0:key})
    table_ = table_.merge(tb_ss, on=[f'ho_type0_{schm}'], how='right')
table_[f'std_lat_{schm}'] = np.sqrt(table_[f'var_lat_{schm}'])

# recv number
tb_recv = tb_group[f'recv_{schm}'].agg(['sum']).reset_index().rename(columns={'sum':f'recv_{schm}'})
table_ = table_.merge(tb_recv, on=[f'ho_type0_{schm}'], how='right')

# jitter
tb_jitter = table[table[f'jitter_{schm}'].notna()].groupby(f'ho_type0_{schm}').apply(weighted_average, f'jitter_{schm}', f'evt_count_{schm}') \
    .reset_index().rename(columns={0:f'jitter_{schm}'})
table_ = table_.merge(tb_jitter, on=[f'ho_type0_{schm}'], how='right')
    
# show table
table_
table_[table_[f'recv_{schm}'] != 0]

,ho_type0_B3,evt_count_B3,mean_lat_B3,median_lat_B3,min_lat_B3,max_lat_B3,std_lat_B3,var_lat_B3,recv_B3,jitter_B3
0,stable,1,0.029646,0.029072,0.005484,0.359905,0.014327,0.000205,230709,NaN
1,LTE_HO,2,0.031910,0.031263,0.028211,0.037273,0.002218,0.000005,706,0.002649
2,MN_HO,18,0.036979,0.033779,0.029326,0.100160,0.015713,0.000247,14962,0.002727
3,SN_HO,15,0.039011,0.037649,0.033674,0.074074,0.006286,0.000040,12299,0.002719
5,SN_Rel,3,0.033401,0.029974,0.024657,0.082676,0.013720,0.000188,1211,0.002772
6,SN_Setup,6,0.051576,0.032094,0.026670,0.173395,0.047131,0.002221,3099,0.002998


## Single Radio Overall

## Dual Radio Loss

### classify by ho_type0

In [21]:
# specify a scheme
schemes = ['B3','B8']
schm1, schm2 = schemes[0], schemes[1]

# count loss
key = f'{schm1}+{schm2}'
table = df.loc[df[f'lost_{key}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{key}'})
table[f'lost_{key}'] = table[f'lost_{key}'].astype('Int16')

key1 = f'{schm1}+^{schm2}'
table1 = df.loc[df[f'lost_{key1}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key1}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'lost_{key1}'] = table[f'lost_{key1}'].astype('Int16')

key2 = f'^{schm1}+{schm2}'
table1 = df.loc[df[f'lost_{key2}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key2}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'lost_{key2}'] = table[f'lost_{key2}'].astype('Int16')

# key3 = f'^{schm1}+^{schm2}'
# table1 = df.loc[df[f'lost_{key3}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
# table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key3}'})
# table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
#     .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
# table[f'lost_{key3}'] = table[f'lost_{key3}'].astype('Int32')

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{key}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)

# calculate loss rate
table[f'lorate(%)_{key}'] = round(table[f'lost_{key}'] / table[f'total_{key}'] * 100, 3)
table[f'lorate(%)_{key1}'] = round(table[f'lost_{key1}'] / table[f'total_{key}'] * 100, 3)
table[f'lorate(%)_{key2}'] = round(table[f'lost_{key2}'] / table[f'total_{key}'] * 100, 3)
# table[f'lorate(%)_{key3}'] = round(table[f'lost_{key3}'] / table[f'total_{key}'] * 100, 3)

# show table 1
table
table[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key2}'] != 0)]

# show table 2
table.sort_values([f'ho_type0_{schm2}', f'ho_type0_{schm1}'])
table[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key2}'] != 0)].sort_values([f'ho_type0_{schm2}', f'ho_type0_{schm1}'])

,lost_B3+B8,ho_type0_B3,ho_type0_B8,lost_B3+^B8,lost_^B3+B8,total_B3+B8,lorate(%)_B3+B8,lorate(%)_B3+^B8,lorate(%)_^B3+B8
2,0,stable,stable,0,166,203313,0.0,0.0,0.082
13,0,MN_HO,stable,333,0,8362,0.0,3.982,0.0
21,0,SN_Setup,stable,117,0,2140,0.0,5.467,0.0
0,0,stable,MN_HO,2,230,17581,0.0,0.011,1.308
15,0,SN_HO,MN_HO,0,10,1422,0.0,0.0,0.703
1,0,stable,SN_HO,1,0,6722,0.0,0.015,0.0
11,0,MN_HO,SN_Rel,0,1,1162,0.0,0.0,0.086
12,0,MN_HO,SN_Setup,0,24,1249,0.0,0.0,1.922
16,0,SN_HO,SN_Setup,0,193,502,0.0,0.0,38.446
18,0,SN_Rel,SN_Setup,0,168,789,0.0,0.0,21.293


## Dual Radio Excessive Latency

### classify by ho_type0

In [22]:
# specify a scheme
schemes = ['B3','B8']
schm1, schm2 = schemes[0], schemes[1]

# count loss
key = f'{schm1}+{schm2}'
table = df.loc[~df[f'lost_{key}'] & df[f'excl_{key}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'excl_{key}'})
table[f'excl_{key}'] = table[f'excl_{key}'].astype('Int16')

key1 = f'{schm1}+^{schm2}'
table1 = df.loc[~df[f'lost_{key1}'] & df[f'excl_{key1}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'excl_{key1}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'excl_{key1}'] = table[f'excl_{key1}'].astype('Int16')

key2 = f'^{schm1}+{schm2}'
table1 = df.loc[~df[f'lost_{key2}'] & df[f'excl_{key2}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'excl_{key2}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'excl_{key2}'] = table[f'excl_{key2}'].astype('Int16')

# key3 = f'^{schm1}+^{schm2}'
# table1 = df.loc[~df[f'lost_{key3}'] & df[f'excl_{key3}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
# table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'excl_{key3}'})
# table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
#     .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
# table[f'excl_{key3}'] = table[f'excl_{key3}'].astype('Int32')

# count total packet
table1 = df.loc[~df[f'lost_{key}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'recv_{key}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
    
table1 = df.loc[~df[f'lost_{key1}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'recv_{key1}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
    
table1 = df.loc[~df[f'lost_{key2}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'recv_{key2}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
    
# table1 = df.loc[~df[f'lost_{key3}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
# table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'recv_{key3}'})
# table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
#     .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)

# calculate loss rate
table[f'exrate(%)_{key}'] = round(table[f'excl_{key}'] / table[f'recv_{key}'] * 100, 3)
table[f'exrate(%)_{key1}'] = round(table[f'excl_{key1}'] / table[f'recv_{key1}'] * 100, 3)
table[f'exrate(%)_{key2}'] = round(table[f'excl_{key2}'] / table[f'recv_{key2}'] * 100, 3)
# table[f'exrate(%)_{key3}'] = round(table[f'excl_{key3}'] / table[f'recv_{key3}'] * 100, 3)

# show table 1
table
table[(table[f'excl_{key1}'] != 0) | (table[f'excl_{key2}'] != 0)]

# show table 2
table.sort_values([f'ho_type0_{schm2}', f'ho_type0_{schm1}'])
table[(table[f'excl_{key1}'] != 0) | (table[f'excl_{key2}'] != 0)].sort_values([f'ho_type0_{schm2}', f'ho_type0_{schm1}'])

,ho_type0_B3,ho_type0_B8,excl_B3+B8,excl_B3+^B8,excl_^B3+B8,recv_B3+B8,recv_B3+^B8,recv_^B3+B8,exrate(%)_B3+B8,exrate(%)_B3+^B8,exrate(%)_^B3+B8
2,stable,stable,0,0,250,203313,203313,203147,0.0,0.0,0.123
8,MN_HO,stable,1,179,68,8362,8029,8362,0.012,2.229,0.813
17,SN_HO,stable,0,0,64,4654,4654,4654,0.0,0.0,1.375
20,SN_Rel,stable,0,0,32,40,40,40,0.0,0.0,80.0
23,SN_Setup,stable,0,233,29,2140,2023,2140,0.0,11.518,1.355
0,stable,MN_HO,0,152,0,17581,17579,17351,0.0,0.865,0.0
13,MN_HO,MN_HO,0,31,0,2822,2822,2822,0.0,1.099,0.0
15,SN_HO,MN_HO,0,0,132,1422,1422,1412,0.0,0.0,9.348
1,stable,SN_HO,0,69,1,6722,6721,6722,0.0,1.027,0.015
10,MN_HO,SN_HO,0,15,121,1490,1490,1490,0.0,1.007,8.121


# Compare B3, B8 (one trace)

## Loss

In [23]:
### B3
# specify a scheme
schemes = ['B3','B8']
schm = schemes[0]

# count loss
table = df.loc[df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{schm}'})

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'lost_{schm}'] = table[f'lost_{schm}'].astype('Int16')

# calculate loss rate
table[f'lorate(%)_{schm}'] = round(table[f'lost_{schm}'] / table[f'total_{schm}'] * 100, 3)

# show table
table_B3 = table[table[f'lost_{schm}'] != 0]


### B8
# specify a scheme
schemes = ['B3','B8']
schm = schemes[1]

# count loss
table = df.loc[df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{schm}'})

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'lost_{schm}'] = table[f'lost_{schm}'].astype('Int16')

# calculate loss rate
table[f'lorate(%)_{schm}'] = round(table[f'lost_{schm}'] / table[f'total_{schm}'] * 100, 3)

# show table
table_B8 = table[table[f'lost_{schm}'] != 0]


### B3+B8
# specify a scheme
schemes = ['B3','B8']
schm1, schm2 = schemes[0], schemes[1]

# count loss
key = f'{schm1}+{schm2}'
table = df.loc[df[f'lost_{key}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{key}'})
table[f'lost_{key}'] = table[f'lost_{key}'].astype('Int16')

key1 = f'{schm1}+^{schm2}'
table1 = df.loc[df[f'lost_{key1}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key1}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'lost_{key1}'] = table[f'lost_{key1}'].astype('Int16')

key2 = f'^{schm1}+{schm2}'
table1 = df.loc[df[f'lost_{key2}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key2}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'lost_{key2}'] = table[f'lost_{key2}'].astype('Int16')

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{key}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)

# calculate loss rate
table[f'lorate(%)_{key}'] = round(table[f'lost_{key}'] / table[f'total_{key}'] * 100, 3)
table[f'lorate(%)_{key1}'] = round(table[f'lost_{key1}'] / table[f'total_{key}'] * 100, 3)
table[f'lorate(%)_{key2}'] = round(table[f'lost_{key2}'] / table[f'total_{key}'] * 100, 3)

# show table 1
# table_B3_B8 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key2}'] != 0),
table_B3_B8 = table.loc[(table[f'lost_{key1}'] != 0),
                  [f'ho_type0_{schm1}', f'ho_type0_{schm2}', f'lost_{key}', f'lost_{key1}', f'total_{key}', f'lorate(%)_{key}', f'lorate(%)_{key1}']]

# show table 2
# table_B8_B3 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key2}'] != 0),
table_B8_B3 = table.loc[(table[f'lost_{key2}'] != 0),
                  [f'ho_type0_{schm2}', f'ho_type0_{schm1}', f'lost_{key}', f'lost_{key2}', f'total_{key}', f'lorate(%)_{key}', f'lorate(%)_{key2}']] \
                    .sort_values([f'ho_type0_{schm2}', f'ho_type0_{schm1}'])

In [24]:
display(table_B3)
display(table_B3_B8)

,ho_type0_B3,lost_B3,total_B3,lorate(%)_B3
0,stable,3,230712,0.001
2,MN_HO,333,15295,2.177
6,SN_Setup,117,3216,3.638


,ho_type0_B3,ho_type0_B8,lost_B3+B8,lost_B3+^B8,total_B3+B8,lorate(%)_B3+B8,lorate(%)_B3+^B8
0,stable,MN_HO,0,2,17581,0.0,0.011
1,stable,SN_HO,0,1,6722,0.0,0.015
13,MN_HO,stable,0,333,8362,0.0,3.982
21,SN_Setup,stable,0,117,2140,0.0,5.467


In [25]:
display(table_B8)
display(table_B8_B3)

,ho_type0_B8,lost_B8,total_B8,lorate(%)_B8
0,stable,166,219718,0.076
2,MN_HO,240,22040,1.089
5,SN_Rel,1,2429,0.041
6,SN_Setup,385,3513,10.959


,ho_type0_B8,ho_type0_B3,lost_B3+B8,lost_^B3+B8,total_B3+B8,lorate(%)_B3+B8,lorate(%)_^B3+B8
2,stable,stable,0,166,203313,0.0,0.082
0,MN_HO,stable,0,230,17581,0.0,1.308
15,MN_HO,SN_HO,0,10,1422,0.0,0.703
11,SN_Rel,MN_HO,0,1,1162,0.0,0.086
12,SN_Setup,MN_HO,0,24,1249,0.0,1.922
16,SN_Setup,SN_HO,0,193,502,0.0,38.446
18,SN_Setup,SN_Rel,0,168,789,0.0,21.293


## Excessive Latency

In [26]:
### B3
# specify a scheme
schemes = ['B3','B8']
schm = schemes[0]

# count loss
table = df.loc[~df[f'lost_{schm}'] & df[f'excl_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'excl_{schm}'})

# count recv packet
table1 = df.loc[~df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'recv_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'excl_{schm}'] = table[f'excl_{schm}'].astype('Int16')

# calculate excessive latency rate
table[f'exrate(%)_{schm}'] = round(table[f'excl_{schm}'] / table[f'recv_{schm}'] * 100, 3)

# show table
table_B3 = table[table[f'excl_{schm}'] != 0]


### B8
# specify a scheme
schemes = ['B3','B8']
schm = schemes[1]

# count loss
table = df.loc[~df[f'lost_{schm}'] & df[f'excl_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'excl_{schm}'})

# count recv packet
table1 = df.loc[~df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'recv_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'excl_{schm}'] = table[f'excl_{schm}'].astype('Int16')

# calculate excessive latency rate
table[f'exrate(%)_{schm}'] = round(table[f'excl_{schm}'] / table[f'recv_{schm}'] * 100, 3)

# show table
table_B8 = table[table[f'excl_{schm}'] != 0]


### B3+B8
# specify a scheme
schemes = ['B3','B8']
schm1, schm2 = schemes[0], schemes[1]

# count loss
key = f'{schm1}+{schm2}'
table = df.loc[~df[f'lost_{key}'] & df[f'excl_{key}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'excl_{key}'})
table[f'excl_{key}'] = table[f'excl_{key}'].astype('Int16')

key1 = f'{schm1}+^{schm2}'
table1 = df.loc[~df[f'lost_{key1}'] & df[f'excl_{key1}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'excl_{key1}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'excl_{key1}'] = table[f'excl_{key1}'].astype('Int16')

key2 = f'^{schm1}+{schm2}'
table1 = df.loc[~df[f'lost_{key2}'] & df[f'excl_{key2}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'excl_{key2}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'excl_{key2}'] = table[f'excl_{key2}'].astype('Int16')

# count total packet
table1 = df.loc[~df[f'lost_{key}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'recv_{key}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
    
table1 = df.loc[~df[f'lost_{key1}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'recv_{key1}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
    
table1 = df.loc[~df[f'lost_{key2}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'recv_{key2}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)

# calculate loss rate
table[f'exrate(%)_{key}'] = round(table[f'excl_{key}'] / table[f'recv_{key}'] * 100, 3)
table[f'exrate(%)_{key1}'] = round(table[f'excl_{key1}'] / table[f'recv_{key1}'] * 100, 3)
table[f'exrate(%)_{key2}'] = round(table[f'excl_{key2}'] / table[f'recv_{key2}'] * 100, 3)

# show table 1
# table_B3_B8 = table.loc[(table[f'excl_{key1}'] != 0) | (table[f'excl_{key2}'] != 0),
table_B3_B8 = table.loc[(table[f'excl_{key1}'] != 0),
                  [f'ho_type0_{schm1}', f'ho_type0_{schm2}', f'excl_{key}', f'excl_{key1}', f'recv_{key}', f'recv_{key1}', f'exrate(%)_{key}', f'exrate(%)_{key1}']]

# show table 2
# table_B8_B3 = table.loc[(table[f'excl_{key1}'] != 0) | (table[f'excl_{key2}'] != 0),
table_B8_B3 = table.loc[(table[f'excl_{key2}'] != 0),
                  [f'ho_type0_{schm2}', f'ho_type0_{schm1}', f'excl_{key}', f'excl_{key2}', f'recv_{key}', f'recv_{key1}', f'exrate(%)_{key}', f'exrate(%)_{key2}']] \
                    .sort_values([f'ho_type0_{schm2}', f'ho_type0_{schm1}'])

In [27]:
display(table_B3)
display(table_B3_B8)

,ho_type0_B3,excl_B3,recv_B3,exrate(%)_B3
0,stable,221,230709,0.096
2,MN_HO,261,14962,1.744
5,SN_Rel,3,1211,0.248
6,SN_Setup,338,3099,10.907


,ho_type0_B3,ho_type0_B8,excl_B3+B8,excl_B3+^B8,recv_B3+B8,recv_B3+^B8,exrate(%)_B3+B8,exrate(%)_B3+^B8
0,stable,MN_HO,0,152,17581,17579,0.0,0.865
1,stable,SN_HO,0,69,6722,6721,0.0,1.027
8,MN_HO,stable,1,179,8362,8029,0.012,2.229
9,MN_HO,SN_Setup,5,30,1249,1249,0.4,2.402
10,MN_HO,SN_HO,0,15,1490,1490,0.0,1.007
13,MN_HO,MN_HO,0,31,2822,2822,0.0,1.099
18,SN_Rel,SN_Setup,2,1,789,789,0.253,0.127
21,SN_Setup,SN_HO,1,31,532,532,0.188,5.827
23,SN_Setup,stable,0,233,2140,2023,0.0,11.518
24,SN_Setup,SN_Setup,0,73,329,329,0.0,22.188


In [28]:
display(table_B8)
display(table_B8_B3)

,ho_type0_B8,excl_B8,recv_B8,exrate(%)_B8
0,stable,444,219552,0.202
2,MN_HO,132,21800,0.606
3,SN_HO,173,14954,1.157
5,SN_Rel,2,2428,0.082
6,SN_Setup,99,3128,3.165


,ho_type0_B8,ho_type0_B3,excl_B3+B8,excl_^B3+B8,recv_B3+B8,recv_B3+^B8,exrate(%)_B3+B8,exrate(%)_^B3+B8
2,stable,stable,0,250,203313,203313,0.0,0.123
8,stable,MN_HO,1,68,8362,8029,0.012,0.813
17,stable,SN_HO,0,64,4654,4654,0.0,1.375
20,stable,SN_Rel,0,32,40,40,0.0,80.0
23,stable,SN_Setup,0,29,2140,2023,0.0,1.355
15,MN_HO,SN_HO,0,132,1422,1422,0.0,9.348
1,SN_HO,stable,0,1,6722,6721,0.0,0.015
10,SN_HO,MN_HO,0,121,1490,1490,0.0,8.121
16,SN_HO,SN_HO,0,21,5632,5632,0.0,0.373
21,SN_HO,SN_Setup,1,29,532,532,0.188,5.451
